In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
from unidecode import unidecode

In [8]:
df_netflix = pd.read_excel('../data/database_netflix_Jan-Jun23_cleanned.xlsx')
df_netflix

,Title,Available Globally?,Release Date,Hours Viewed,Category,Season nb
0,The Night Agent,Yes,2023-03-23,812100000,Serie,1.0
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000,Serie,NaN
2,The Glory,Yes,2022-12-30,622800000,Serie,1.0
3,Wednesday,Yes,2022-11-23,507700000,Serie,1.0
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000,Serie,NaN
...,...,...,...,...,...,...
18188,ZombieLars: Season 2,No,NaT,100000,Serie,NaN
18189,Zone 261,No,NaT,100000,NaN,NaN
18190,Zulu,No,NaT,100000,NaN,NaN
18191,[REC] (2007),No,NaT,100000,Movie,NaN


In [9]:
df_netflix_test = df_netflix.copy()

In [10]:
df_netflix_test['Title_clean'] = df_netflix_test['Title'].str.extract(r'^(.*?)(?:: Season \d+)?$')[0]
df_netflix_test['Season nb2'] = df_netflix_test['Season nb'].combine_first(df_netflix_test['Title'].str.extract(r'Season (\d+)$')[0])

df_netflix_test

,Title,Available Globally?,Release Date,Hours Viewed,Category,Season nb,Title_clean,Season nb2
0,The Night Agent,Yes,2023-03-23,812100000,Serie,1.0,The Night Agent,1.0
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000,Serie,NaN,Ginny & Georgia,2
2,The Glory,Yes,2022-12-30,622800000,Serie,1.0,The Glory,1.0
3,Wednesday,Yes,2022-11-23,507700000,Serie,1.0,Wednesday,1.0
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000,Serie,NaN,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...,...,...,...,...,...
18188,ZombieLars: Season 2,No,NaT,100000,Serie,NaN,ZombieLars,2
18189,Zone 261,No,NaT,100000,NaN,NaN,Zone 261,NaN
18190,Zulu,No,NaT,100000,NaN,NaN,Zulu,NaN
18191,[REC] (2007),No,NaT,100000,Movie,NaN,[REC] (2007),NaN


In [11]:
df_netflix_test['Title_clean'].isna().sum()

0

In [59]:
df_program_list = pd.read_excel('../data/programme_list.xlsx')
df_program_list

,official_title
0,The Night Agent: Season 1
1,Ginny & Georgia: Season 2
2,The Glory: Season 1 // 더 글로리: 시즌 1
3,Wednesday: Season 1
4,Queen Charlotte: A Bridgerton Story
...,...
18189,ZombieLars: Season 2
18190,Zone 261 // Operation Ragnarok
18191,Zulu
18192,[REC] (2007)


In [60]:
df_program_list[['Title_en', 'Title_translated']] = df_program_list['official_title'].apply(lambda x: pd.Series(str(x).split('//', 1)))
df_program_list

,official_title,Title_en,Title_translated
0,The Night Agent: Season 1,The Night Agent: Season 1,NaN
1,Ginny & Georgia: Season 2,Ginny & Georgia: Season 2,NaN
2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory: Season 1,더 글로리: 시즌 1
3,Wednesday: Season 1,Wednesday: Season 1,NaN
4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...
18189,ZombieLars: Season 2,ZombieLars: Season 2,NaN
18190,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok
18191,Zulu,Zulu,NaN
18192,[REC] (2007),[REC] (2007),NaN


In [61]:
df_program_list[['Title_eng', 'drop']] = df_program_list['Title_en'].apply(lambda x: pd.Series(str(x).split(': Season ', 1)))

In [65]:
df_program_list = df_program_list[['official_title','Title_eng', 'Title_translated']]
df_program_list

,official_title,Title_eng,Title_translated
0,The Night Agent: Season 1,The Night Agent,NaN
1,Ginny & Georgia: Season 2,Ginny & Georgia,NaN
2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory,더 글로리: 시즌 1
3,Wednesday: Season 1,Wednesday,NaN
4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...
18189,ZombieLars: Season 2,ZombieLars,NaN
18190,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok
18191,Zulu,Zulu,NaN
18192,[REC] (2007),[REC] (2007),NaN


In [80]:
df_program_list['official_title'][532]


'Cobra Kai: Season 1'

In [79]:
cobrakai = df_program_list[df_program_list['official_title'] == 'Cobra Kai: Season 1']
cobrakai

,official_title,Title_eng,Title_translated
532,Cobra Kai: Season 1,Cobra Kai,NaN


In [89]:
df_program_list = df_program_list.drop([532])
df_program_list

,official_title,Title_eng,Title_translated
0,The Night Agent: Season 1,The Night Agent,NaN
1,Ginny & Georgia: Season 2,Ginny & Georgia,NaN
2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory,더 글로리: 시즌 1
3,Wednesday: Season 1,Wednesday,NaN
4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...
18189,ZombieLars: Season 2,ZombieLars,NaN
18190,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok
18191,Zulu,Zulu,NaN
18192,[REC] (2007),[REC] (2007),NaN


In [90]:
df_netflix_test.head(3)

,Title,Available Globally?,Release Date,Hours Viewed,Category,Season nb,Title_clean,Season nb2
0,The Night Agent,Yes,2023-03-23,812100000,Serie,1.0,The Night Agent,1.0
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000,Serie,NaN,Ginny & Georgia,2
2,The Glory,Yes,2022-12-30,622800000,Serie,1.0,The Glory,1.0


In [95]:
df_program_list['index'] = range(len(df_program_list))
df_program_list


,official_title,Title_eng,Title_translated,index
0,The Night Agent: Season 1,The Night Agent,NaN,0
1,Ginny & Georgia: Season 2,Ginny & Georgia,NaN,1
2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory,더 글로리: 시즌 1,2
3,Wednesday: Season 1,Wednesday,NaN,3
4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN,4
...,...,...,...,...
18189,ZombieLars: Season 2,ZombieLars,NaN,18188
18190,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok,18189
18191,Zulu,Zulu,NaN,18190
18192,[REC] (2007),[REC] (2007),NaN,18191


In [96]:
df_netflix_test['index'] = range(len(df_netflix_test))

In [99]:
df_netflix_test.columns

Index(['Title', 'Available Globally?', 'Release Date', 'Hours Viewed',
       'Category', 'Season nb', 'Title_clean', 'Season nb2', 'index'],
      dtype='object')

In [101]:
df_netflix = df_netflix_test[['index', 'Title_clean', 'Season nb2', 'Category', 'Hours Viewed', 
                              'Available Globally?', 'Release Date']]

In [103]:
df_netflix = df_netflix.rename(columns={'Title_clean': 'Original_Title', 'Season nb2': 'Season nb'})
df_netflix

,index,Original_Title,Season nb,Category,Hours Viewed,Available Globally?,Release Date
0,0,The Night Agent,1.0,Serie,812100000,Yes,2023-03-23
1,1,Ginny & Georgia,2,Serie,665100000,Yes,2023-01-05
2,2,The Glory,1.0,Serie,622800000,Yes,2022-12-30
3,3,Wednesday,1.0,Serie,507700000,Yes,2022-11-23
4,4,Queen Charlotte: A Bridgerton Story,NaN,Serie,503000000,Yes,2023-05-04
...,...,...,...,...,...,...,...
18188,18188,ZombieLars,2,Serie,100000,No,NaT
18189,18189,Zone 261,NaN,NaN,100000,No,NaT
18190,18190,Zulu,NaN,NaN,100000,No,NaT
18191,18191,[REC] (2007),NaN,Movie,100000,No,NaT


In [104]:
df_program_list.columns

Index(['official_title', 'Title_eng', 'Title_translated', 'index'], dtype='object')

In [107]:
df_program_list = df_program_list[['index', 'official_title', 'Title_eng', 'Title_translated']]

In [108]:
df_program_list

,index,official_title,Title_eng,Title_translated
0,0,The Night Agent: Season 1,The Night Agent,NaN
1,1,Ginny & Georgia: Season 2,Ginny & Georgia,NaN
2,2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory,더 글로리: 시즌 1
3,3,Wednesday: Season 1,Wednesday,NaN
4,4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...,...
18189,18188,ZombieLars: Season 2,ZombieLars,NaN
18190,18189,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok
18191,18190,Zulu,Zulu,NaN
18192,18191,[REC] (2007),[REC] (2007),NaN


In [111]:
netflix_final_db = pd.merge(df_netflix, df_program_list, on='index', how='inner')
netflix_final_db


,index,Original_Title,Season nb,Category,Hours Viewed,Available Globally?,Release Date,official_title,Title_eng,Title_translated
0,0,The Night Agent,1.0,Serie,812100000,Yes,2023-03-23,The Night Agent: Season 1,The Night Agent,NaN
1,1,Ginny & Georgia,2,Serie,665100000,Yes,2023-01-05,Ginny & Georgia: Season 2,Ginny & Georgia,NaN
2,2,The Glory,1.0,Serie,622800000,Yes,2022-12-30,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory,더 글로리: 시즌 1
3,3,Wednesday,1.0,Serie,507700000,Yes,2022-11-23,Wednesday: Season 1,Wednesday,NaN
4,4,Queen Charlotte: A Bridgerton Story,NaN,Serie,503000000,Yes,2023-05-04,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,NaN
...,...,...,...,...,...,...,...,...,...,...
18188,18188,ZombieLars,2,Serie,100000,No,NaT,ZombieLars: Season 2,ZombieLars,NaN
18189,18189,Zone 261,NaN,NaN,100000,No,NaT,Zone 261 // Operation Ragnarok,Zone 261,Operation Ragnarok
18190,18190,Zulu,NaN,NaN,100000,No,NaT,Zulu,Zulu,NaN
18191,18191,[REC] (2007),NaN,Movie,100000,No,NaT,[REC] (2007),[REC] (2007),NaN


In [162]:
netflix_final_db.to_csv('netflix_final_db.csv', index=False)